<a href="https://colab.research.google.com/github/JINZhao2000/PanGenAI/blob/main/panGenAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install graphdatascience -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.0/919.0 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 KB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from graphdatascience import GraphDataScience

host = "bolt://47.254.177.216:7687"
user = "neo4j"
password = "PanGenAI2023"

gds = GraphDataScience(host, auth=(user, password))

In [3]:
# Test the connectivity
gds.run_cypher(
 """ MATCH (n) RETURN labels(n)[0] AS label, count(*) AS count"""
)

,label,count
0,Gene,3334670
1,Family,90872
2,Module,3459
3,RGP,32177
4,Spot,1077
5,Contig,35678
6,Genome,728
7,Pangenome,5
8,Partition,3


In [4]:

G_nom, results = gds.graph.project("fam", "Family", 
                                   {"NEIGHBOR": {"orientation": "UNDIRECTED"},
                                    "IS_SIMILAR": {"orientation": "UNDIRECTED"}
                                    })

In [5]:
results

nodeProjection            {'Family': {'label': 'Family', 'properties': {}}}
relationshipProjection    {'IS_SIMILAR': {'orientation': 'UNDIRECTED', '...
graphName                                                               fam
nodeCount                                                             90872
relationshipCount                                                    733128
projectMillis                                                           362
Name: 0, dtype: object

In [6]:
gds.wcc.stats(G_nom)['componentDistribution']

{'p99': 48,
 'min': 1,
 'max': 89512,
 'mean': 199.71868131868132,
 'p90': 5,
 'p50': 1,
 'p999': 89512,
 'p95': 11,
 'p75': 2}

----We create a graph only with the connected families we call it *G_connected*

In [7]:
#Preform the louvain algorithm
gds.louvain.write(G_nom, writeProperty = "louvainCommunity")


Louvain:   0%|          | 0/100 [00:00<?, ?%/s]

writeMillis                                                            381
nodePropertiesWritten                                                90872
modularity                                                         0.94171
modularities             [0.6680063202213442, 0.8622299059832368, 0.926...
ranLevels                                                                5
communityCount                                                         644
communityDistribution    {'p99': 1554, 'min': 1, 'max': 4125, 'mean': 1...
postProcessingMillis                                                     6
preProcessingMillis                                                      0
computeMillis                                                         9888
configuration            {'maxIterations': 10, 'writeConcurrency': 4, '...
Name: 0, dtype: object

In [8]:
#Preform the label propagation algorithm
gds.labelPropagation.write(G_nom, writeProperty = "labelPropCommunity")

LabelPropagation:   0%|          | 0/100 [00:00<?, ?%/s]

writeMillis                                                            128
nodePropertiesWritten                                                90872
ranIterations                                                           10
didConverge                                                          False
communityCount                                                       18084
communityDistribution    {'p99': 27, 'min': 1, 'max': 1653, 'mean': 5.0...
postProcessingMillis                                                    24
preProcessingMillis                                                      0
computeMillis                                                          799
configuration            {'jobId': '9f8b9663-65cb-4db8-b48f-68b5a2f8b7d...
Name: 0, dtype: object

In [10]:
gds.wcc.mutate(G_nom, mutateProperty = "wccComponent")
#component means community

mutateMillis                                                             1
nodePropertiesWritten                                                90872
componentCount                                                         455
componentDistribution    {'p99': 48, 'min': 1, 'max': 89512, 'mean': 19...
postProcessingMillis                                                     8
preProcessingMillis                                                      0
computeMillis                                                           41
configuration            {'jobId': 'fafd780b-bc55-4dfb-9b82-1dab834c5ec...
Name: 0, dtype: object